In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [ ]:
# tf.function and autograph
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3., -2.5])))

scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print(scaled_elu_tf(tf.constant([-3., -2.5])))

print(scaled_elu_tf.python_function is scaled_elu)

In [ ]:
%timeit scaled_elu(tf.random.normal((1000, 1000)))
%timeit scaled_elu_tf(tf.random.normal((1000, 1000)))

In [ ]:
# 1 + 1/2 + 1/2^2 + ... + 1/2^n

@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(20))

In [ ]:
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))

In [ ]:
display_tf_code(scaled_elu)

In [ ]:
# display_tf_code(converge_to_2) # don't know why this is not working

In [ ]:
var = tf.Variable(0.)

@tf.function
def add_21():
#     var = tf.Variable(0.) # not able to define inside tensorflow function
    return var.assign_add(21)

print(add_21())

In [ ]:
# @tf.function py func -> graph
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)

try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)

print(cube(tf.constant([1, 2, 3])))

In [ ]:
# get_concrete_function -> add input signature -> SavedModel

cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None], tf.int32))
print(cube_func_int32)

In [ ]:
print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5], tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(tf.constant([1, 2, 3])))

In [ ]:
cube_func_int32.graph

In [ ]:
cube_func_int32.graph.get_operations()

In [ ]:
pow_op = cube_func_int32.graph.get_operations()[2]
print(pow_op)

In [ ]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))

In [ ]:
cube_func_int32.graph.get_operation_by_name("x")

In [ ]:
cube_func_int32.graph.get_tensor_by_name("x:0")

In [ ]:
cube_func_int32.graph.as_graph_def()